In [1]:
import clingo
from gamearg import *
from IPython.display import display, Image
import pandas as pd

## Generating Graph 

With given `node_num`, program will be able to generate all possible edges

In [2]:
def graph_generating(node_num):
    def on_model(m):
        if str(m) == "":
            possible_graphs.append(str(m))
        else:
            elements = str(m).split(" ")
            elements_with_dot = [element + "." for element in elements]
            concatenated = " ".join(elements_with_dot)
            possible_graphs.append(concatenated)

    graph_log = """
    #const n = {}.
    node(1..n).
    {{move(X,Y): node(X), node(Y)}}.
    #show move/2.
    """.format(node_num)
    # print(graph_log)
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.add("base", [], graph_log)
    ctl.ground([("base", [])])
    possible_graphs = []
    ctl.solve(on_model=on_model)
    return possible_graphs

In [3]:
possible_graphs = graph_generating(node_num=2)

In [4]:
possible_graphs

['',
 'move(1,2).',
 'move(2,1).',
 'move(1,2). move(2,1).',
 'move(2,2).',
 'move(2,1). move(2,2).',
 'move(1,2). move(2,2).',
 'move(1,2). move(2,1). move(2,2).',
 'move(1,1).',
 'move(1,1). move(1,2).',
 'move(1,1). move(2,2).',
 'move(1,1). move(1,2). move(2,2).',
 'move(1,1). move(2,1).',
 'move(1,1). move(2,1). move(2,2).',
 'move(1,1). move(1,2). move(2,1).',
 'move(1,1). move(1,2). move(2,1). move(2,2).']

## Digest into Game Solver

Based on the `possible_graphs`, we need to digest them into game_solver one by one and ensure that the result will be 3-valued (dir=back and forward)

In [5]:
columns = ["node_number", "edge_number", "graph", "fw_node_wfs", "bw_node_wfs"]
graph_cal = pd.DataFrame(columns=columns)

node_num = 1
file_path = "files/temp.lp"
while node_num<4:
    possible_graphs = graph_generating(node_num)
    if not possible_graphs or all(graph == "" for graph in possible_graphs):
        node_num += 1
        continue
    else:
        for graph in possible_graphs:
            if graph == "":
                continue
            else:
                try:
                    with open(file_path, "w") as file:
                        file.write(graph)
                    fw_node_wfs_df = node_wfs_cal(file_path, "game", reverse=False)
                    bw_node_wfs_df = node_wfs_cal(file_path, "game", reverse=True)
                    
                    #store the new_data into the dataframe
                    new_data = pd.DataFrame({
                        "node_number": node_num,
                        "edge_number": len(graph.split(".")),
                        "graph": graph,
                        "fw_node_wfs": [set(fw_node_wfs_df["wfs"])],
                        "bw_node_wfs": [set(bw_node_wfs_df["wfs"])]
                    })
                    graph_cal = pd.concat([graph_cal, new_data], ignore_index=True)
                    if (
                        len(set(fw_node_wfs_df["wfs"])) == 3
                        and len(set(bw_node_wfs_df["wfs"])) == 3
                    ):
                        print("Here is the node number for M3-both: " + str(node_num))
                        print(graph)
    
                except Exception as e:
                    print(f"An error occurred while processing {graph}: {e}")

        node_num += 1

Here is the node number for M3-both: 3
move(1,3). move(2,2).
Here is the node number for M3-both: 3
move(2,2). move(3,1).
Here is the node number for M3-both: 3
move(1,2). move(3,3).
Here is the node number for M3-both: 3
move(2,1). move(3,3).
Here is the node number for M3-both: 3
move(1,1). move(3,2).
Here is the node number for M3-both: 3
move(1,1). move(2,3).
